In [1]:
# import libraries
import pandas as pd
import sklearn
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
# import rpy2
# import met
import os

In [78]:
# import data from source
# - https://covidtracking.com/about-data/data-definitions
df = pd.read_csv('all_states.csv', low_memory = False)

In [87]:
# Change all numerical columns to integers
num_cols = df.columns.tolist()[3:]
for n in num_cols:
    df[n] = df[n].apply(lambda x: int(x) if x != np.NaN)

SyntaxError: invalid syntax (<ipython-input-87-2ee582c3dbe9>, line 4)

#### Create Dataset
 - Each observation is a day, each entity is a state

In [51]:
# Select features of interest
feature = 'positiveIncrease'
# first, second, second_last, last

In [72]:
df.head()

,date,state,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,12/24/2020,AK,A,199.0,199.0,2,NaN,983.0,983.0,105.0,...,1227735.0,5708,NaN,NaN,NaN,NaN,NaN,0,1227735.0,5708
1,12/24/2020,AL,A,4676.0,4093.0,89,583.0,31954.0,31954.0,2542.0,...,1824681.0,11330,NaN,NaN,84371.0,NaN,1824681.0,11330,NaN,0
2,12/24/2020,AR,A+,3406.0,2946.0,30,460.0,10879.0,10879.0,1093.0,...,1989073.0,18218,NaN,21856.0,NaN,221177.0,NaN,0,1989073.0,18218
3,12/24/2020,AS,D,0.0,NaN,0,NaN,NaN,NaN,NaN,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,12/24/2020,AZ,A+,8294.0,7538.0,115,756.0,34520.0,34520.0,4221.0,...,4787230.0,54963,386490.0,NaN,NaN,NaN,2720517.0,22502,4787230.0,54963


In [66]:
# Create dataset 
states = df['state'].unique().tolist()

data = df.loc[df['state'] == states[0], ['date', feature]]
data.rename(columns = {feature: states[0]}, inplace = True)

for s in states[1:3]:
    sub = df.loc[df['state'] == s, ['date', feature]]
    sub.rename(columns = {feature: s}, inplace = True)
    sub[s + '_first'] = sub[s].apply(lambda x: int(str(x)[:1]) if x != 0 else np.NaN)
    sub[s + '_second'] = sub[s].apply(lambda x: int(str(x)[1:2]) if len(str(x)) > 1 else np.NaN)
    sub[s + '_second_last'] = sub[s].apply(lambda x: int(str(x)[-2:-1]) if len(str(x)) > 1 else np.NaN)
    sub[s + '_last'] = sub[s].apply(lambda x: int(str(x)[-1]) if x != 0 else np.NaN)
    data = data.merge(sub, how = 'left', on = 'date')

ValueError: invalid literal for int() with base 10: '-'

In [11]:
# dataset of first digit distributions
for row, i in data.iterrows():
    

,date,state,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,12/24/2020,AK,A,199.0,199.0,2,NaN,983.0,983.0,105.0,...,1227735.0,5708,NaN,NaN,NaN,NaN,NaN,0,1227735.0,5708
1,12/24/2020,AL,A,4676.0,4093.0,89,583.0,31954.0,31954.0,2542.0,...,1824681.0,11330,NaN,NaN,84371.0,NaN,1824681.0,11330,NaN,0
2,12/24/2020,AR,A+,3406.0,2946.0,30,460.0,10879.0,10879.0,1093.0,...,1989073.0,18218,NaN,21856.0,NaN,221177.0,NaN,0,1989073.0,18218
3,12/24/2020,AS,D,0.0,NaN,0,NaN,NaN,NaN,NaN,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,12/24/2020,AZ,A+,8294.0,7538.0,115,756.0,34520.0,34520.0,4221.0,...,4787230.0,54963,386490.0,NaN,NaN,NaN,2720517.0,22502,4787230.0,54963


In [2]:
np.random.seed(123)
ex = np.random.randint(low=1, high=10, size=9).tolist()

# Benford First Digit = log10(1+1/n)
ref = list(np.log10(1 + 1/np.arange(1, 10)))

In [3]:
len(ex), len(ref)

(9, 9)

In [4]:
t = pd.DataFrame({'obs': [ex], 'ref': [ref], 'sum': sum(ex)})
t['ref_adj'] = [[x * t['sum'][0] for x in t['ref'].tolist()[0]]]

In [7]:
m = met.Multinom(t['ref_adj'][0], t['obs'][0])

In [ ]:
res = m.twosided_exact_test(save_cases = False)

In [ ]:
res